训练集共包括5个城市，每个城市目录下的数据集总体说明：

1.各区域每天新增感染人数。文件名：infection.csv。提供前45天每天数据，文件格式为日期,区域ID,新增感染人数；“,”分割。

2.城市间迁徙指数。文件名：migration.csv。提供45天每天数据。文件格式为迁徙日期,迁徙出发城市,迁徙到达城市,迁徙指数；“,”分割。

3.网格人流量指数。文件名：density.csv。提供45天内每周两天抽样数据，文件格式为日期，小时，网格中心点经度,网格中心点纬度,人流量指数；“,”分割。

4.网格关联强度。文件名：transfer.csv。城市内网格间关联强度数据，文件格式为小时，出发网格中心点经度，出发网格中心点纬度，到达网格中心点经度，到达网格中心点纬度，迁移强度；“,”分割。

5.网格归属区域。文件名：grid_attr.csv。城市内网格对应的归属区域ID，文件格式为网格中心点经度，网格中心点纬度，归属区域ID；“,”分割。

6.天气数据。文件名：weather.csv。提供45天每天数据，文件格式为日期,小时,气温,湿度,风向,风速,风力,天气；“,”分割。



# Baseline方法
## 数据处理+训练数据构造
首先城市之间是不能独立处理的，因为多个城市之间具有相互迁移的动作，具体字段在migration里面。
如果用ML/DL的方法来回归预测新增感染人数，那么我们的训练集的每个样本应当具有下列属性：
- 当前日期，当前区域新增感染人数：这个作为训练的标签
- 城市
- 日期
- 区域ID
- 过去N个日期(N可以定义为潜伏期长度)其余城市迁移到该城市的人口，即输入人口(来自于**migration**数据)以及“其余城市”过去的感染数
- 前一个日期（或者前N个）的新增感染人数（**infection数据**）
- 前面N个日期的天气指标（利用到**weather**数据）
- 过去N个日期该区域的人流量密度(利用到**density**数据)
- 区域之间的迁移强度（**transfer**数据）以及和本区域有迁移关系的其它区域的感染情况

**注**：

- 因为density,transfer的地理信息表示都是以经纬度的方式来表示而并不是直接以区域id的方式给出，所以这里就需要用到**grid_attr**数据来将经纬度信息与区域ID信息联系起来。
- 不同表之间的数据大都可以通过每张表的**日期**数据进行关联。

以上就是一个数据处理基本思路，以及所给6个数据文件的作用浅析。利用python的pandas库可以很好的实现以上思路，主要的方法是利用
数据表之间的连接，利用共同字段对表进行连接操作！

### pandas具体操作
- 11个城市，每个城市6张表，总共30张表。需要不同表之间交互的其实只有migration表
- 表与表之间的连接需要日期和ID
- 经纬度转区域ID的操作不太好搞，数据量太大,尝试了先groupby数据，


## 回归预测方法（scikit-learn库）
### xgboost&lightgbm&catboost
### nn

# 读取5个城市的训练数据
Solution()这个类的的作用是读取训练数据，总共5个城市，按照pandas,dataframe的格式读出来，并且赋予列名称。然后所有的数据都存储在一个字典中

self.dic_data={

'city_A':{'transfer', 'density', 'migration', 'infection', 'weather', 'grid_attr'}, 

'city_B':{transfer', 'density', 'migration', 'infection', 'weather', 'grid_attr''}, 

'city_C':{transfer', 'density', 'migration', 'infection', 'weather', 'grid_attr''},

'city_D':{transfer', 'density', 'migration', 'infection', 'weather', 'grid_attr''},

'city_E':{transfer', 'density', 'migration', 'infection', 'weather', 'grid_attr''}

}

In [1]:
import os
import numpy as np
import pandas as pd
import pickle

In [8]:

class Solotion(object):
    def __init__(self,train_data_dir):
        self.train_data_dir=train_data_dir
        self.dic_data={}
    #
    def read_train_data(self):
        city_names=sorted(os.listdir(self.train_data_dir))
        for city_name in city_names:
            if city_name[-4:]=='.csv':
                continue
            print('开始读取%s的数据.........:'% city_name)
            city_path=os.path.join(self.train_data_dir,city_name)#分别读取5个城市
            self.read_perCity(city_name,city_path)
            print('%s的数据读取完毕.........:'% city_name)
    #
    def read_perCity(self,city_name,city_path):
        #city_ID=city_name[-1]  #A,B,C....
        data={}
        transfer=pd.read_csv(os.path.join(city_path,'transfer.csv'),header=None)
        transfer.columns = ['hour', 'start_grid_x','start_grid_y','end_grid_x','end_grid_y','index']
        #
        density=pd.read_csv(os.path.join(city_path,'density.csv'),header=None)
        density.columns = ['date', 'hour','grid_x','grid_y','index']
        #
        migration=pd.read_csv(os.path.join(city_path,'migration.csv'),header=None)
        migration.columns = ['date', 'departure_city','arrival_city','index']
        #
        infection=pd.read_csv(os.path.join(city_path,'infection.csv'),header=None)
        infection.columns = ['city','region_id','date', 'index']
        #
        weather=pd.read_csv(os.path.join(city_path,'weather.csv'),header=None)
        weather.columns = ['data','hour', 'temperature','humidity','wind_direction','wind_speed','wind_force','weather']
        #
        grid_attr=pd.read_csv(os.path.join(city_path,'grid_attr.csv'),header=None)
        grid_attr.columns = ['grid_x', 'grid_y','region_id']
        #加入字典data
        data['transfer']=transfer
        data['density']=density
        data['migration']=migration
        data['infection']=infection
        data['weather']=weather
        data['grid_attr']=grid_attr
        #汇总到总字典
        self.dic_data[city_name]=data


In [9]:
s=Solotion('E:/迅雷下载/train_data_semi_final_round/')#
s.read_train_data()

开始读取city_A的数据.........:
city_A的数据读取完毕.........:
开始读取city_B的数据.........:
city_B的数据读取完毕.........:
开始读取city_C的数据.........:
city_C的数据读取完毕.........:
开始读取city_D的数据.........:
city_D的数据读取完毕.........:
开始读取city_E的数据.........:
city_E的数据读取完毕.........:
开始读取city_F的数据.........:
city_F的数据读取完毕.........:
开始读取city_G的数据.........:
city_G的数据读取完毕.........:
开始读取city_H的数据.........:
city_H的数据读取完毕.........:
开始读取city_I的数据.........:
city_I的数据读取完毕.........:
开始读取city_J的数据.........:
city_J的数据读取完毕.........:
开始读取city_K的数据.........:
city_K的数据读取完毕.........:


In [10]:
train_data=s.dic_data #train-data存储了所有的数据
train_data.keys(),train_data['city_A'].keys()

(dict_keys(['city_A', 'city_B', 'city_C', 'city_D', 'city_E', 'city_F', 'city_G', 'city_H', 'city_I', 'city_J', 'city_K']),
 dict_keys(['transfer', 'density', 'migration', 'infection', 'weather', 'grid_attr']))

In [11]:
train_data['city_A']['transfer'].head(2)#验证一下输出

,hour,start_grid_x,start_grid_y,end_grid_x,end_grid_y,index
0,0,145.453431,29.461634,145.45706,29.477293,0.1
1,0,145.453431,29.461634,145.45706,29.477293,0.3


In [12]:
#序列化存储和读取数据
'''
import pickle
output_pkl = open('/home/aistudio/work/train_data.pkl', 'wb')
# Pickle dictionary using protocol 0.
pickle.dump(train_data, output_pkl)
'''
pkl_file = open('/home/aistudio/work/train_data.pkl', 'rb')

train_data = pickle.load(pkl_file)


"\npkl_file = open('train_data.pkl', 'rb')\n\ntrain_data = pickle.load(pkl_file)\n"

## 数据预处理

In [ ]:
'''
#处理数据时，可以以infection表为中心
#处理city_A的数据
city_A_data=train_data['city_A']
transfer=city_A_data['transfer']
density=city_A_data['density']
migration=city_A_data['density']
infection=city_A_data['infection']
weather=city_A_data['weather']
grid_attr=city_A_data['grid_attr']
print(density.date.unique())#只有12天的密度信息
print(len(grid_attr.grid_x.unique()),len(grid_attr.grid_y.unique()))
print(len(density.grid_x.unique()),len(density.grid_y.unique()))
print(len(transfer.start_grid_x.unique()),len(transfer.start_grid_y.unique()),len(transfer.end_grid_x.unique()),len(transfer.end_grid_y.unique()))
'''

从以上三个结果可以看出，经纬度乱七八糟，还是需要先转换成ID才好进行表之间的连接

为了对不同的表做连接，感觉需要先将经度纬度坐标转换为区域ID，具体的分配准则可以采取最近邻法则，给定一个经纬度的坐标，

只需要将这个坐标分别和区域id的中心坐标求距离，最近的距离分配给该经纬度

In [1]:

# #下面代码的主要作用在于处理density和tranfer数据:将坐标都从属于区域id
# for city_id in ['A','B','C','D','E','F','G','H','I','J','K']:
#     #处理数据时，可以以infection表为中心
#     #处理city_A的数据
#     city_data=train_data['city_'+city_id]
#     transfer=city_data['transfer']
#     density=city_data['density']
#     migration=city_data['migration']
#     infection=city_data['infection']
#     weather=city_data['weather']
#     grid_attr=city_data['grid_attr']
#     #譬如对于cityA总共118个区域ID,找到每一个id的中心
#     grouped_id=grid_attr.groupby(['region_id'])
#     region_center=grouped_id['grid_x','grid_y'].agg(np.mean).reset_index()
#     #----------------将经纬度转换成对应的区域ID的函数--------------------
#     #平均处理一条数据要
#     def xy_to_id(x,y):
#         distance={}
#         for index,Id,x0,y0 in region_center.itertuples():
#             distance[index]=(x-x0)*(x-x0)+(y-y0)*(y-y0)
#         distance=sorted(distance.items(),key=lambda item:item[1]) #按照距离排序，取最近的index进行匹配
#         return distance[0][0] #返回该经纬度对应的区域Id
#     #----------------处理density(35906205)数据--------------------
#     '''
#     含有经,纬度数据的表(density和transfer)全部换成区域ID数据，并且把经度，纬度drop掉,
#     对于density和transfer数据,如果不对原始数据进行处理，直接转换id速度太慢无法接受，
#     感觉可以把小时信息给消除掉，这样可以极大的消减数据量,譬如可以尝试按照经纬度groupby然后取1整天的平均密度
#     '''
#     #----------------对density的12个日期按照经纬度groupby，然后取一天的平均密度--------------------
#     grouped=density.groupby(['date'])#这样会group成12组数据
#     #[21200506, 21200509, 21200513, 21200516, 21200520, 21200523,21200527, 21200530, 21200603, 21200606, 21200610, 21200613]
#     dates=density.date.unique()#得到这12组数据的日期编号：21200506,.....,21200613
#     density_dic={}#把density切分成12个部分，分别处理存放
#     for per in dates:
#         print("开始处理日期%d的数据"%per)
#         d=grouped.get_group(per) #per:21200506.....,共12天
#         d_xy=d.groupby(['grid_x','grid_y']) #按照经纬度进行group
#         mean_index=d_xy['index'].agg(np.mean).reset_index()
#         density_dic[per]=mean_index
#     print("density数据的groupby完成.............")
#     #print(density_dic[21200506].head())#可以打印看一下21200506这天的数据


#     #----------------依次处理12天的数据:将经纬度替换为区域ID(...)--------------------

#     #即使经过了groupyby，每一天还是会有大约19w条经纬度组合，但是12天的数据大都是重复的，所以
#     menmory_dic={} #为了节约时间，计算过的经纬度不再重复计算，而是用一个字典来实现记忆
#     for per in dates:
#         print("开始为日期%d的数据转换ID"%per)
#         temp=density_dic[per][['grid_x','grid_y']]
#         re_list=[]
#         for index,x,y in temp.itertuples():
#             str_xy=str(x)+str(y)
#             if str_xy in menmory_dic:#判断是不是已经计算过这个经纬度组合，如果计算过，直接从字典中取
#                 re_list.append(menmory_dic[str_xy])
#                 continue
#             re_id=xy_to_id(x,y)#该经纬度对应的区域ID
#             re_list.append(re_id)
#             menmory_dic[str_xy]=re_id
#         density_dic[per]['region_id']=re_list #把转换后的id列添加到dataframe中
#         density_dic[per].drop(['grid_x','grid_y'],axis=1,inplace=True)
#         grouped_density=density_dic[per].groupby(['region_id'])
#         density_dic[per]=grouped_density['index'].agg(np.mean).reset_index()

#     #每一个density_dic[per]里面都存储了一个dataframe代表了per这一天各个区域的density,还可以对它按照区域ID做一个groupby
#     #--------------------为了不用每次都跑，将结果存入pickle中---------------------

#     import pickle
#     output_pkl = open('dataset/density_'+city_id+'.pkl', 'wb')
#     # Pickle dictionary using protocol 0.
#     pickle.dump(density_dic, output_pkl)

#     '''
#     pkl_file = open('/home/aistudio/work/density_A.pkl', 'rb')
    
#     density = pickle.load(pkl_file)
#     '''
#     #----------------处理transfer(5670548)（...）数据--------------------
#     grouped_trans=transfer.groupby(['start_grid_x','start_grid_y','end_grid_x','end_grid_y'])
#     mean_index_trans=grouped_trans['index'].agg(np.mean).reset_index()
#     start_list=[]
#     end_list=[]
#     for i,sx,sy,ex,ey,_ in mean_index_trans.itertuples():
#         if i%500000==0:#打印处理进度
#             print(i)
#         start_xy=str(sx)+str(sy)
#         end_xy=str(ex)+str(ey)
#         if start_xy in menmory_dic :#判断是不是已经计算过这个经纬度组合，如果计算过，直接从字典中取
#             start_list.append(menmory_dic[start_xy])
#         else:
#             start_id=xy_to_id(sx,sy)#该经纬度对应的区域ID
#             start_list.append(start_id)
#             menmory_dic[start_xy]=start_id
#         if end_xy in menmory_dic:
#                 end_list.append(menmory_dic[end_xy])
#         else:
#             end_id=xy_to_id(ex,ey)#该经纬度对应的区域ID
#             end_list.append(end_id)
#             menmory_dic[end_xy]=end_id
#     mean_index_trans['start_id']=start_list
#     mean_index_trans['end_id']=end_list
#     mean_index_trans.drop(['start_grid_x','start_grid_y','end_grid_x','end_grid_y'],axis=1,inplace=True)
#     grouped_tranId=mean_index_trans.groupby(['start_id','end_id'])
#     write_tranId=grouped_tranId['index'].agg(np.sum).reset_index()
#     write_tranId.to_csv('dataset/transferId_'+city_id+'.csv',index=False)
#     #transferId_A里面存储了城市A各个区域内部的连接强度和外部的连接强度





## 构造特征


### 构造density特征
不能单纯的pd.merge(),因为desnsity是和日期有关的，对于特定的一个样本，需要先找在该样本日期之前的density抽样的那个日期。然后将
那个日期对应的该样本的区域的平均density返回
### 构造transfer特征
对于某一个区域id：有内部的tranfer也有id之间的transfer,所以这里可以构造两个特征,例如对于id为0的区域:
- 那么(0,0)的transfer总和代表了它内部的强度
- (1,0)+(2,0)+....+(i,0)+....代表了外部流向0的强度
### 构造weather特征，
构造的weather特征需要是当前日期前N天的数据，N可以定义为潜伏期；预处理weather数据，去掉小时信息，以天为单位

In [18]:
#------------------把migration数据合并一下--------------
migration_A=train_data['city_A']['migration']
migration_B=train_data['city_B']['migration']
migration_C=train_data['city_C']['migration']
migration_D=train_data['city_D']['migration']
migration_E=train_data['city_E']['migration']
migration_F=train_data['city_F']['migration']
migration_G=train_data['city_G']['migration']
migration_H=train_data['city_H']['migration']
migration_I=train_data['city_I']['migration']
migration_J=train_data['city_J']['migration']
migration_K=train_data['city_K']['migration']
migration_merge=pd.concat([migration_A, migration_B,migration_C, migration_D,migration_E,
                           migration_F, migration_G,migration_H, migration_I,migration_J,migration_K]).reset_index(drop=True)

potential=12 #潜伏期
for city_id in ['A','B','C','D','E','F','G','H','I','J','K']:
    print("----处理city_%s的数据"%city_id)
    city_data = train_data['city_' + city_id]
    transfer = city_data['transfer']
    density = city_data['density']
    migration = city_data['migration']
    infection = city_data['infection']
    weather = city_data['weather']
    grid_attr = city_data['grid_attr']
    #----------第一步是要整合infection和grid_attr的信息------------
    df_train=pd.merge(infection, grid_attr, left_on=['region_id'], right_on=['region_id'], how='left')
    df_train.drop(['grid_x','grid_y'],axis=1,inplace=True)
    df_train.drop_duplicates(keep='first',inplace=True)
    df_train.reset_index(drop=True,inplace=True)
    #df_train.head(2)

    #------------------处理weather特征，构造的weather特征需要是当前日期前N天的数据，N可以定义为潜伏期--------------
    #----------预处理weather数据，去掉小时信息，以天为单位-------------
    #返回每个object一天出现最多的情况
    def func(df):
        tmp=df.tolist()
        return max(tmp,key=tmp.count)
    weather=weather.fillna(0)#首先填充缺省值
    #对天气按照data进行groupby,将hour信息去掉,这里的date(日期)拼错为了data,但不影响，以后再改了
    wea_grouped=weather.groupby(['data'])
    wea_temp=wea_grouped['temperature'].agg(np.mean).reset_index()
    wea_humi=wea_grouped['humidity'].apply(func).to_frame().reset_index()
    wea_dire=wea_grouped['wind_direction'].apply(func).to_frame().reset_index()
    wea_speed=wea_grouped['wind_speed'].apply(func).to_frame().reset_index()
    wea_force=wea_grouped['wind_force'].apply(func).to_frame().reset_index()
    wea_type=wea_grouped['weather'].apply(func).to_frame().reset_index()

    #---------------构造density特征和transfer特征------------------
    date_list=[21200506, 21200509, 21200513, 21200516, 21200520, 21200523,
    21200527, 21200530, 21200603, 21200606, 21200610, 21200613]
    df_trans=pd.read_csv('dataset/transferId_'+city_id+'.csv')#存储了区域内部和外部的transfer特征
    #构造density特征：找到当前样本日期之前的density抽样日期
    pkl_file = open('dataset/density_'+city_id+'.pkl', 'rb')
    density_dic = pickle.load(pkl_file)
    def get_densityFeature(date):
        for i in range(12):
            if date_list[i]>date:
                return date_list[i-1]
        return date_list[11]
    #构造transfer特征
    def get_transFeature(region_id):
        iner=0#内部强度
        ext=0#外部强度
        for _,start_id,end_id,index in df_trans.itertuples():
            if end_id==region_id and start_id==region_id:
                iner=iner+index
            if end_id==region_id:
                ext=ext+index
        return iner,ext

    #构造weather特征
    def get_weatherFeature(date):
        date_list=weather['data'].unique()#45天
        for i in range(len(date_list)):
            if date_list[i]>date:
                break
        if i<potential:
            tmp_date=[date_list[j] for j in range(i)]
        else:
            tmp_date=[date_list[j] for j in range(i-potential,i)]#取潜伏期这么长度的时间段计算天气状况

        temper_list=[]#温度
        humi_list=[]#湿度
        dire_list=[]#风向
        speed_list=[]#风速
        force_list=[]#风力
        wtype_list=[]#类型
        for per_d in tmp_date:
            temper_list.append(wea_temp[wea_temp['data']==per_d]['temperature'].reset_index(drop=True)[0])
            humi_list.append(wea_humi[wea_humi['data']==per_d]['humidity'].reset_index(drop=True)[0])
            dire_list.append(wea_dire[wea_dire['data']==per_d]['wind_direction'].reset_index(drop=True)[0])
            speed_list.append(wea_speed[wea_speed['data']==per_d]['wind_speed'].reset_index(drop=True)[0])
            force_list.append(wea_force[wea_force['data']==per_d]['wind_force'].reset_index(drop=True)[0])
            wtype_list.append(wea_type[wea_type['data']==per_d]['weather'].reset_index(drop=True)[0])
        #前N天出现最多的天气种类
        re1=np.mean(temper_list)#温度取平均值，而不是众数
        re2=max(humi_list,key=humi_list.count)
        re3=max(dire_list,key=dire_list.count)
        re4=max(speed_list,key=speed_list.count)
        re5=max(force_list,key=force_list.count)
        re6=max(wtype_list,key=wtype_list.count)
        return  re1,re2,re3,re4,re5,re6

    #构造migration特征
    #譬如对于城市A来说，先计算出a的当前日期的前N天内迁移到A的迁移指数总和
    def get_migrateFeature(date,city_name):
        date_list=weather['data'].unique()#45天
        for i in range(len(date_list)):
            if date_list[i]>date:
                break
        if i<potential:
            tmp_date=[date_list[j] for j in range(i)]
        else:
            tmp_date=[date_list[j] for j in range(i-potential,i)]#取潜伏期这么长度的时间段计算城市间迁移状况
        re=0
        for _,per_date,dep,arr,value in migration_merge.itertuples():
            if arr==city_name and (per_date in tmp_date):
                re+=value
        return re
    #
    density_value=[]#密度
    iner_transfer=[]#内部强度
    ext_transfer=[]#外部强度
    temper=[]#温度
    humi=[]#湿度
    dire=[]#风向
    speed=[]#风速
    force=[]#风力
    wtype=[]#类型
    mig_list=[]#城市之间迁移特征
    for _,_,region_id,date,_ in df_train.itertuples():
        #构造density特征
        loc_date=get_densityFeature(date)
        density_value.append(density_dic[loc_date]['index'][region_id])
        #构造transfer特征
        iner,ext=get_transFeature(region_id)
        iner_transfer.append(iner)
        ext_transfer.append(ext)
        #构造weather特征
        re1,re2,re3,re4,re5,re6=get_weatherFeature(date)
        temper.append(re1)
        humi.append(re2)
        dire.append(re3)
        speed.append(re4)
        force.append(re5)
        wtype.append(re6)
        #构造migration 特征
        mig_list.append(get_migrateFeature(date,city_id))

    df_train['density']=density_value
    df_train['iner_transfer']=iner_transfer
    df_train['ext_transfer']=ext_transfer
    df_train['temperature']=temper
    df_train['humidity']=humi
    df_train['wind_direction']=dire
    df_train['wind_speed']=speed
    df_train['wind_force']=force
    df_train['weather_type']=wtype
    df_train['migration']=mig_list
    #df_train.head(2)
    df_train.to_csv('dataset/features/'+'features_'+city_id+'.csv',index=False)

----处理city_A的数据
----处理city_B的数据
----处理city_C的数据
----处理city_D的数据
----处理city_E的数据
----处理city_F的数据
----处理city_G的数据
----处理city_H的数据
----处理city_I的数据
----处理city_J的数据
----处理city_K的数据
